In [1]:
import os
import cv2


In [2]:
from keras.preprocessing import image

In [3]:
categories = ['with_mask','without_mask']

In [4]:
data=[]
for category in categories :
    path = os.path.join('train',category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])    
        

In [5]:
print(len(data))

1950


In [6]:
import random 

In [7]:
random.shuffle(data)

In [8]:
X= []
y=[]

for features,label in data:
    X.append(features)
    y.append(label)
    

In [9]:
len(X)

1950

In [10]:
len(y)

1950

In [11]:
import numpy as np


In [12]:
X=np.array(X)
y=np.array(y)
# print(X)
X.shape

(1950, 224, 224, 3)

In [13]:
y

array([1, 0, 0, ..., 0, 1, 0])

In [14]:
X = X/255
# print(X)

In [15]:
# from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
# train_test_split()

In [17]:
X_train.shape

(1560, 224, 224, 3)

In [18]:
X_test.shape

(390, 224, 224, 3)

In [19]:
from keras.applications.vgg16 import VGG16

In [20]:
vgg = VGG16()

In [21]:
vgg.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
# removing last layer and adding ours


In [23]:
from keras import Sequential

In [24]:
model = Sequential(vgg.layers[:-1])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [26]:
for layer in model.layers:
    layer.trainable=False

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:
from keras.layers import Dense

In [29]:
model.add(Dense(1,activation='sigmoid'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [31]:
model.compile(optimizer='Adam',loss ='binary_crossentropy',metrics =['accuracy'])

In [32]:
model.fit(X_train,y_train,epochs = 3 , validation_data=(X_test,y_test))

Epoch 1/3
49/49 [==============================] - 373s 8s/step - loss: 0.3902 - accuracy: 0.8596 - val_loss: 0.2464 - val_accuracy: 0.9513
Epoch 2/3
49/49 [==============================] - 390s 8s/step - loss: 0.1749 - accuracy: 0.9763 - val_loss: 0.1484 - val_accuracy: 0.9718
Epoch 3/3
49/49 [==============================] - 448s 9s/step - loss: 0.1244 - accuracy: 0.9821 - val_loss: 0.1116 - val_accuracy: 0.9744


In [33]:
model.save('train_k_model.h5')